# NBA Game Predictor
### CMPE 257 Project
Authors: Kaushika Uppu, Miranda Billawala, Yun Ei Hlaing, Iris Cheung

## Imports

In [145]:
import pandas as pd
import numpy as np
import time
from nba_api.stats.static import teams
from nba_api.stats.endpoints import teamgamelogs

## Loading In NBA Game Data

In [14]:
nba_teams = teams.get_teams()
all_teamids = []
for team in nba_teams:
    all_teamids.append(team['id'])

In [112]:
def get_all_nba_seasons(start_year = 1985, end_year = None):
    if end_year is None:
        end_year = datetime.datetime.now().year
    
    seasons = []
    for year in range(start_year, end_year + 1):
        seasons.append(f"{year}-{str(year + 1)[2:]}")
    
    return seasons

In [115]:
all_seasons = get_all_nba_seasons(end_year = 2023)

In [116]:
def get_team_logs(team_id, season):
    team_log = teamgamelogs.TeamGameLogs(team_id_nullable = team_id, season_nullable = season)
    games = team_log.get_data_frames()[0]
    return games

In [117]:
dfs = []
for t_id in all_teamids:
    for season in all_seasons:
        curr_game_logs = get_team_logs(t_id, season)
        if curr_game_logs is not None:
            dfs.append(curr_game_logs)
        time.sleep(.600)

In [185]:
all_stats = pd.concat(dfs, ignore_index = True)

/var/folders/46/8chr9vpj0zg39crspv4tbrp80000gn/T/ipykernel_66336/1154032993.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_stats = pd.concat(dfs, ignore_index = True)


In [186]:
all_stats.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK,AVAILABLE_FLAG
0,21985,1610612737,ATL,Atlanta Hawks,0028500933,1986-04-12,ATL vs. IND,W,240.0,38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,21985,1610612737,ATL,Atlanta Hawks,0028500921,1986-04-10,ATL vs. NJN,W,240.0,44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,21985,1610612737,ATL,Atlanta Hawks,0028500908,1986-04-08,ATL vs. CHI,W,240.0,52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,21985,1610612737,ATL,Atlanta Hawks,0028500891,1986-04-05,ATL @ CHI,L,240.0,40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,21985,1610612737,ATL,Atlanta Hawks,0028500884,1986-04-04,ATL @ WAS,L,265.0,54,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [187]:
len(all_stats[all_stats['SEASON_YEAR'].isna()])

23370

In [190]:
# getting all the data from nba_api takes a while, exporting to csv so we can just import data
all_stats.to_csv('all_game_stats.csv', index = False)

In [191]:
# all_stats = pd.read_csv('all_game_stats.csv')

### Inputting Missing Season Values

In [192]:
for index, row in all_stats.iterrows():
    if pd.isna(all_stats.iloc[index]['SEASON_YEAR']):
        year_index = all_stats.iloc[index]['GAME_DATE'].find('-')
        year = all_stats.iloc[index]['GAME_DATE'][:year_index]
        month = all_stats.iloc[index]['GAME_DATE'][year_index+1:year_index+3]
        if int(month) >= 10:
            season = str(int(year)) + "-" + str(int(year)+1)[2:]
        else:
            season = str(int(year)-1) + "-" + str(int(year))[2:]
        all_stats.loc[index, 'SEASON_YEAR'] = season

In [193]:
len(all_stats[all_stats['SEASON_YEAR'].isna()])

0

In [194]:
all_stats.isna().sum()

SEASON_ID            66172
TEAM_ID                  0
TEAM_ABBREVIATION        0
TEAM_NAME                0
GAME_ID                  0
GAME_DATE                0
MATCHUP                  0
WL                       0
MIN                      0
FGM                      0
FGA                      0
FG_PCT                   0
FG3M                     0
FG3A                     0
FG3_PCT                475
FTM                      0
FTA                      0
FT_PCT                   0
OREB                     0
DREB                     0
REB                      0
AST                      0
STL                      0
BLK                      0
TOV                      0
PF                       0
PTS                      0
PLUS_MINUS               0
VIDEO_AVAILABLE      66172
SEASON_YEAR              0
BLKA                 23370
PFD                  23370
GP_RANK              23370
W_RANK               23370
L_RANK               23370
W_PCT_RANK           23370
MIN_RANK             23370
F